# 🚴 Predictive Maintenance for Rental Bike Fleets
## Using Usage Pattern Analysis

**Team**: Anomaly Archers (Section B)

**Techniques Used**:
- K-Means Clustering (Unsupervised Learning)
- Isolation Forest (Anomaly Detection)
- Time Series Analysis
- Health Scoring & Maintenance Ranking

---

## 1. Setup & Imports

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML libraries
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

print("✅ All libraries imported successfully!")

## 2. Data Loading & Exploration

In [ ]:
# Load both months of data
print("Loading data...")

df_nov = pd.read_csv('../data/raw/202411-capitalbikeshare-tripdata.csv')
df_dec = pd.read_csv('../data/raw/202412-capitalbikeshare-tripdata.csv')

# Combine into single dataframe
df = pd.concat([df_nov, df_dec], ignore_index=True)

print(f"✅ Loaded {len(df):,} trip records")
print(f"   November: {len(df_nov):,} trips")
print(f"   December: {len(df_dec):,} trips")

In [ ]:
# View the data structure
print("\n📊 Dataset Overview:")
print(f"Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\n Data Types:")
print(df.dtypes)

In [ ]:
# Sample of the data
df.head()

In [ ]:
# Check for missing values
print("\n🔍 Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(2)
missing_df = pd.DataFrame({'Missing': missing, 'Percentage': missing_pct})
print(missing_df[missing_df['Missing'] > 0])

In [ ]:
# Identify the bike ID column
# Looking for columns that might contain bike identifiers
print("\n🔑 Looking for Bike ID column...")
for col in df.columns:
    print(f"{col}: {df[col].nunique()} unique values")

## 3. Data Cleaning & Preprocessing

In [ ]:
# Convert datetime columns
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

# Calculate trip duration in minutes
df['duration_min'] = (df['ended_at'] - df['started_at']).dt.total_seconds() / 60

print("✅ Datetime columns converted")
print(f"\nDuration statistics (minutes):")
print(df['duration_min'].describe())

In [ ]:
# Remove invalid trips (negative or extremely long durations)
print(f"\nBefore cleaning: {len(df):,} trips")

# Filter: duration between 1 minute and 180 minutes (3 hours)
df_clean = df[(df['duration_min'] >= 1) & (df['duration_min'] <= 180)].copy()

print(f"After cleaning: {len(df_clean):,} trips")
print(f"Removed: {len(df) - len(df_clean):,} trips ({(len(df) - len(df_clean))/len(df)*100:.2f}%)")

In [ ]:
# Check which column contains bike IDs
# Usually it's 'rideable_type' combined with 'ride_id' or there might be a bike number
print("\n🔍 Checking rideable_type distribution:")
print(df_clean['rideable_type'].value_counts())

In [ ]:
# Since this dataset uses ride_id (unique per trip) but we need bike-level analysis,
# we'll use a combination of start_station + rideable_type as a proxy for fleet segments
# OR if there's a bike_id column, we'll use that

# Check all columns again
print("All columns:")
for col in df_clean.columns:
    print(f"  - {col}")

In [ ]:
# IMPORTANT: If there's no direct bike_id, we'll create synthetic bike segments
# This is a valid approach when individual bike IDs aren't available

# Check if there's a bike number or similar column
bike_cols = [col for col in df_clean.columns if 'bike' in col.lower() or 'id' in col.lower()]
print(f"Potential bike ID columns: {bike_cols}")

# Show sample values
for col in bike_cols:
    print(f"\n{col} samples:")
    print(df_clean[col].head(10))

## 4. Feature Engineering

We'll create bike-level features for our analysis. If individual bike IDs aren't available, we'll use station-based analysis as a valid proxy.

In [ ]:
# Strategy: Aggregate by start_station_id to create "station health profiles"
# This is a valid approach when bike-level data isn't available

# First, let's see what station columns we have
station_cols = [col for col in df_clean.columns if 'station' in col.lower()]
print(f"Station columns: {station_cols}")

In [ ]:
# Create aggregated features per station
# This will serve as our "entity" for health analysis

# Group by start station
station_features = df_clean.groupby('start_station_id').agg(
    total_trips=('ride_id', 'count'),
    total_duration_min=('duration_min', 'sum'),
    avg_trip_duration=('duration_min', 'mean'),
    std_trip_duration=('duration_min', 'std'),
    max_trip_duration=('duration_min', 'max'),
    min_trip_duration=('duration_min', 'min'),
    first_trip=('started_at', 'min'),
    last_trip=('started_at', 'max'),
    unique_destinations=('end_station_id', 'nunique'),
    member_trips=('member_casual', lambda x: (x == 'member').sum()),
    casual_trips=('member_casual', lambda x: (x == 'casual').sum())
).reset_index()

print(f"✅ Created features for {len(station_features)} stations")
station_features.head()

In [ ]:
# Calculate additional derived features
station_features['days_active'] = (station_features['last_trip'] - station_features['first_trip']).dt.days + 1
station_features['trips_per_day'] = station_features['total_trips'] / station_features['days_active']
station_features['member_ratio'] = station_features['member_trips'] / station_features['total_trips']
station_features['duration_variability'] = station_features['std_trip_duration'] / station_features['avg_trip_duration']

# Handle any NaN values
station_features = station_features.fillna(0)

print("✅ Derived features calculated")
station_features.head()

In [ ]:
# Filter out stations with very few trips (noise)
min_trips = 50  # Minimum trips to be included
station_features_filtered = station_features[station_features['total_trips'] >= min_trips].copy()

print(f"Stations before filter: {len(station_features)}")
print(f"Stations after filter (>={min_trips} trips): {len(station_features_filtered)}")

In [ ]:
# Select features for ML
feature_columns = [
    'total_trips',
    'total_duration_min',
    'avg_trip_duration',
    'std_trip_duration',
    'trips_per_day',
    'unique_destinations',
    'member_ratio',
    'duration_variability'
]

X = station_features_filtered[feature_columns].copy()

# Handle any infinities
X = X.replace([np.inf, -np.inf], 0)
X = X.fillna(0)

print(f"\n📊 Feature Matrix Shape: {X.shape}")
print(f"\nFeature Statistics:")
X.describe()

In [ ]:
# Normalize features for clustering
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("✅ Features normalized using StandardScaler")
print(f"\nScaled feature statistics:")
print(f"Mean: {X_scaled.mean(axis=0).round(2)}")
print(f"Std:  {X_scaled.std(axis=0).round(2)}")

## 5. K-Means Clustering

We'll segment stations into different usage profiles.

In [ ]:
# Find optimal K using Elbow Method
inertias = []
silhouette_scores = []
K_range = range(2, 8)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X_scaled, kmeans.labels_))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Elbow plot
axes[0].plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[0].set_ylabel('Inertia', fontsize=12)
axes[0].set_title('Elbow Method for Optimal K', fontsize=14)
axes[0].grid(True)

# Silhouette plot
axes[1].plot(K_range, silhouette_scores, 'go-', linewidth=2, markersize=8)
axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)
axes[1].set_ylabel('Silhouette Score', fontsize=12)
axes[1].set_title('Silhouette Score vs K', fontsize=14)
axes[1].grid(True)

plt.tight_layout()
plt.savefig('../outputs/figures/elbow_silhouette.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nSilhouette Scores: {dict(zip(K_range, [f'{s:.3f}' for s in silhouette_scores]))}")

In [ ]:
# Train final K-Means with K=3
K = 3
kmeans = KMeans(n_clusters=K, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)

# Add cluster labels to data
station_features_filtered['cluster'] = cluster_labels

print(f"✅ K-Means clustering complete with K={K}")
print(f"\nCluster Distribution:")
print(station_features_filtered['cluster'].value_counts().sort_index())

In [ ]:
# Analyze cluster characteristics
cluster_summary = station_features_filtered.groupby('cluster')[feature_columns].mean()
print("\n📊 Cluster Characteristics (Mean Values):")
cluster_summary.round(2)

In [ ]:
# Name the clusters based on characteristics
# Analyze which cluster has highest/lowest values
cluster_names = {}

# Find cluster with highest trips per day (Heavy Usage)
heavy_cluster = cluster_summary['trips_per_day'].idxmax()
# Find cluster with lowest trips per day (Light Usage)
light_cluster = cluster_summary['trips_per_day'].idxmin()
# Remaining is Moderate
moderate_cluster = [c for c in range(K) if c not in [heavy_cluster, light_cluster]][0]

cluster_names = {
    heavy_cluster: '🔴 Heavy Usage',
    moderate_cluster: '🟡 Moderate Usage',
    light_cluster: '🟢 Light Usage'
}

station_features_filtered['cluster_name'] = station_features_filtered['cluster'].map(cluster_names)

print("\n📍 Cluster Naming:")
for k, v in cluster_names.items():
    print(f"  Cluster {k}: {v}")

In [ ]:
# Visualize clusters using PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

plt.figure(figsize=(12, 8))

colors = ['#2ecc71', '#f1c40f', '#e74c3c']  # Green, Yellow, Red
for cluster_id in sorted(station_features_filtered['cluster'].unique()):
    mask = station_features_filtered['cluster'] == cluster_id
    plt.scatter(
        X_pca[mask, 0], 
        X_pca[mask, 1], 
        c=colors[cluster_id],
        label=cluster_names[cluster_id],
        alpha=0.6,
        s=100,
        edgecolors='black',
        linewidth=0.5
    )

plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)', fontsize=12)
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)', fontsize=12)
plt.title('Station Clusters - Usage Profiles (PCA Visualization)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.savefig('../outputs/figures/cluster_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nPCA Explained Variance: {pca.explained_variance_ratio_.sum()*100:.1f}%")

## 6. Anomaly Detection (Isolation Forest)

Identify stations with unusual usage patterns that may indicate problems.

In [ ]:
# Train Isolation Forest
iso_forest = IsolationForest(
    contamination=0.05,  # Expect 5% anomalies
    random_state=42,
    n_estimators=100
)

# Fit and predict
anomaly_labels = iso_forest.fit_predict(X_scaled)
anomaly_scores = iso_forest.decision_function(X_scaled)

# Add to dataframe (-1 = anomaly, 1 = normal)
station_features_filtered['anomaly'] = anomaly_labels
station_features_filtered['anomaly_score'] = anomaly_scores
station_features_filtered['is_anomaly'] = station_features_filtered['anomaly'] == -1

print("✅ Isolation Forest trained")
print(f"\nAnomaly Detection Results:")
print(f"  Normal stations: {(anomaly_labels == 1).sum()}")
print(f"  Anomalous stations: {(anomaly_labels == -1).sum()}")
print(f"  Anomaly rate: {(anomaly_labels == -1).mean()*100:.1f}%")

In [ ]:
# Analyze anomalies
anomalies = station_features_filtered[station_features_filtered['is_anomaly']]
normal = station_features_filtered[~station_features_filtered['is_anomaly']]

print("\n📊 Anomaly vs Normal Comparison:")
comparison = pd.DataFrame({
    'Anomaly (Mean)': anomalies[feature_columns].mean(),
    'Normal (Mean)': normal[feature_columns].mean(),
    'Difference %': ((anomalies[feature_columns].mean() - normal[feature_columns].mean()) / normal[feature_columns].mean() * 100)
})
comparison.round(2)

In [ ]:
# Visualize anomalies
plt.figure(figsize=(12, 8))

# Normal points
plt.scatter(
    X_pca[~station_features_filtered['is_anomaly'], 0],
    X_pca[~station_features_filtered['is_anomaly'], 1],
    c='#3498db',
    label='Normal',
    alpha=0.5,
    s=80
)

# Anomaly points
plt.scatter(
    X_pca[station_features_filtered['is_anomaly'], 0],
    X_pca[station_features_filtered['is_anomaly'], 1],
    c='#e74c3c',
    label='Anomaly',
    alpha=0.9,
    s=150,
    marker='X',
    edgecolors='black',
    linewidth=1
)

plt.xlabel('PC1', fontsize=12)
plt.ylabel('PC2', fontsize=12)
plt.title('Anomaly Detection Results (Isolation Forest)', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.savefig('../outputs/figures/anomaly_detection.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Show top anomalous stations
print("\n🚨 Top 10 Most Anomalous Stations:")
top_anomalies = station_features_filtered.nsmallest(10, 'anomaly_score')[
    ['start_station_id', 'cluster_name', 'total_trips', 'trips_per_day', 'avg_trip_duration', 'anomaly_score']
]
top_anomalies

## 7. Time Series Analysis

Analyze usage trends over time to detect degradation patterns.

In [ ]:
# Create daily aggregates
df_clean['date'] = df_clean['started_at'].dt.date

daily_stats = df_clean.groupby(['start_station_id', 'date']).agg(
    daily_trips=('ride_id', 'count'),
    daily_duration=('duration_min', 'sum'),
    avg_duration=('duration_min', 'mean')
).reset_index()

print(f"✅ Created daily aggregates: {len(daily_stats):,} rows")
daily_stats.head()

In [ ]:
# Calculate trends per station (simple linear regression slope)
from scipy import stats

def calculate_trend(group):
    if len(group) < 7:  # Need at least a week of data
        return 0
    x = np.arange(len(group))
    y = group['daily_trips'].values
    slope, _, _, _, _ = stats.linregress(x, y)
    return slope

station_trends = daily_stats.groupby('start_station_id').apply(calculate_trend).reset_index()
station_trends.columns = ['start_station_id', 'usage_trend']

print("\n📈 Usage Trend Statistics:")
print(station_trends['usage_trend'].describe())

In [ ]:
# Merge trends with main features
# IMPORTANT: Drop columns if they exist to avoid merge errors on re-run
cols_to_drop = [c for c in ['usage_trend', 'trend_category', 'usage_trend_x', 'usage_trend_y'] 
               if c in station_features_filtered.columns]
if cols_to_drop:
    station_features_filtered = station_features_filtered.drop(columns=cols_to_drop)

station_features_filtered = station_features_filtered.merge(
    station_trends, 
    on='start_station_id', 
    how='left'
)
station_features_filtered['usage_trend'] = station_features_filtered['usage_trend'].fillna(0)

# Categorize trends
station_features_filtered['trend_category'] = pd.cut(
    station_features_filtered['usage_trend'],
    bins=[-np.inf, -0.5, 0.5, np.inf],
    labels=['📉 Declining', '➡️ Stable', '📈 Increasing']
)

print("\n📊 Trend Distribution:")
print(station_features_filtered['trend_category'].value_counts())

In [ ]:
# Visualize overall fleet trend
fleet_daily = df_clean.groupby('date').agg(
    total_trips=('ride_id', 'count'),
    avg_duration=('duration_min', 'mean')
).reset_index()

fleet_daily['date'] = pd.to_datetime(fleet_daily['date'])
fleet_daily['rolling_7d'] = fleet_daily['total_trips'].rolling(7).mean()

plt.figure(figsize=(14, 6))
plt.plot(fleet_daily['date'], fleet_daily['total_trips'], alpha=0.3, label='Daily Trips')
plt.plot(fleet_daily['date'], fleet_daily['rolling_7d'], linewidth=2, color='red', label='7-Day Rolling Avg')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Number of Trips', fontsize=12)
plt.title('Fleet-Wide Daily Trip Volume', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('../outputs/figures/time_series_trend.png', dpi=150, bbox_inches='tight')
plt.show()

## 8. Health Scoring & Maintenance Ranking

Combine all signals into a unified health score.

In [ ]:
# Define health score components

# 1. Cluster Risk Score (Heavy usage = higher risk)
cluster_risk_map = {
    heavy_cluster: 0.7,
    moderate_cluster: 0.4,
    light_cluster: 0.1
}
station_features_filtered['cluster_risk'] = station_features_filtered['cluster'].map(cluster_risk_map)

# 2. Normalize Anomaly Score (0 to 1, where 1 = most anomalous)
min_score = station_features_filtered['anomaly_score'].min()
max_score = station_features_filtered['anomaly_score'].max()
station_features_filtered['normalized_anomaly'] = 1 - ((station_features_filtered['anomaly_score'] - min_score) / (max_score - min_score))

# 3. Trend Risk Score - convert categorical to string to avoid TypeError
station_features_filtered['trend_category_str'] = station_features_filtered['trend_category'].astype(str)
trend_risk_map = {
    '📉 Declining': 0.8,
    '➡️ Stable': 0.3,
    '📈 Increasing': 0.1
}
station_features_filtered['trend_risk'] = station_features_filtered['trend_category_str'].map(trend_risk_map).fillna(0.3)

print("✅ Risk components calculated")

In [ ]:
# Calculate composite health score
# Weights: Cluster 40%, Anomaly 40%, Trend 20%
w_cluster = 0.4
w_anomaly = 0.4
w_trend = 0.2

station_features_filtered['health_score'] = (
    w_cluster * station_features_filtered['cluster_risk'] +
    w_anomaly * station_features_filtered['normalized_anomaly'] +
    w_trend * station_features_filtered['trend_risk']
)

print("\n📊 Health Score Distribution:")
print(station_features_filtered['health_score'].describe())

In [ ]:
# Assign health categories
def categorize_health(score):
    if score < 0.35:
        return '🟢 Stable'
    elif score < 0.55:
        return '🟡 Warning'
    else:
        return '🔴 Critical'

station_features_filtered['health_category'] = station_features_filtered['health_score'].apply(categorize_health)

print("\n🏥 Health Category Distribution:")
print(station_features_filtered['health_category'].value_counts())

In [ ]:
# Create maintenance priority ranking
priority_ranking = station_features_filtered.sort_values('health_score', ascending=False)[
    ['start_station_id', 'cluster_name', 'health_score', 'health_category', 
     'total_trips', 'trips_per_day', 'is_anomaly', 'trend_category']
].reset_index(drop=True)

priority_ranking.index = priority_ranking.index + 1  # 1-based ranking
priority_ranking.index.name = 'Priority'

print("\n🔧 TOP 20 STATIONS FOR MAINTENANCE:")
priority_ranking.head(20)

In [ ]:
# Save the full ranking
priority_ranking.to_csv('../outputs/maintenance_priority_ranking.csv')
print("✅ Priority ranking saved to outputs/maintenance_priority_ranking.csv")

## 9. Visualizations & Summary

In [ ]:
# Health Score Distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
colors = {'🟢 Stable': '#2ecc71', '🟡 Warning': '#f1c40f', '🔴 Critical': '#e74c3c'}
health_counts = station_features_filtered['health_category'].value_counts()
plt.bar(health_counts.index, health_counts.values, color=[colors[c] for c in health_counts.index])
plt.xlabel('Health Category', fontsize=12)
plt.ylabel('Number of Stations', fontsize=12)
plt.title('Fleet Health Distribution', fontsize=14)

plt.subplot(1, 2, 2)
plt.hist(station_features_filtered['health_score'], bins=20, edgecolor='black', alpha=0.7, color='#3498db')
plt.axvline(x=0.35, color='green', linestyle='--', label='Stable threshold')
plt.axvline(x=0.55, color='orange', linestyle='--', label='Warning threshold')
plt.xlabel('Health Score', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Health Score Histogram', fontsize=14)
plt.legend()

plt.tight_layout()
plt.savefig('../outputs/figures/health_distribution.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Top 10 Priority Bar Chart
top10 = priority_ranking.head(10)

plt.figure(figsize=(12, 6))
colors = ['#e74c3c' if cat == '🔴 Critical' else '#f1c40f' for cat in top10['health_category']]
bars = plt.barh(range(len(top10)), top10['health_score'], color=colors)
plt.yticks(range(len(top10)), [f"Station {sid}" for sid in top10['start_station_id']])
plt.xlabel('Health Score (Higher = More Urgent)', fontsize=12)
plt.title('🔧 Top 10 Stations Requiring Maintenance', fontsize=14)
plt.gca().invert_yaxis()

# Add score labels
for i, (score, cat) in enumerate(zip(top10['health_score'], top10['health_category'])):
    plt.text(score + 0.01, i, f'{score:.2f}', va='center', fontsize=10)

plt.tight_layout()
plt.savefig('../outputs/figures/top10_priority.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Fleet Health Pie Chart
plt.figure(figsize=(8, 8))
health_counts = station_features_filtered['health_category'].value_counts()
colors = [colors.get(c, '#95a5a6') for c in health_counts.index]
plt.pie(
    health_counts.values, 
    labels=health_counts.index, 
    autopct='%1.1f%%',
    colors=['#2ecc71', '#f1c40f', '#e74c3c'][:len(health_counts)],
    explode=[0.05] * len(health_counts),
    shadow=True,
    startangle=90
)
plt.title('Overall Fleet Health Status', fontsize=14)
plt.savefig('../outputs/figures/fleet_health_pie.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Conclusions & Business Insights

### Key Findings

1. **Clustering Results**: Stations were segmented into 3 distinct usage profiles:
   - Heavy Usage: High-traffic stations with intense utilization
   - Moderate Usage: Average activity stations
   - Light Usage: Low-traffic stations

2. **Anomaly Detection**: ~5% of stations show unusual usage patterns that warrant investigation. These anomalies often correlate with:
   - Extremely high or low trip durations
   - Unusual member/casual ratios
   - Inconsistent daily patterns

3. **Time Series Trends**: Identified stations with declining usage that may indicate:
   - Equipment issues discouraging riders
   - Location-based problems
   - Seasonal variations

4. **Health Scoring**: Combined all signals into actionable maintenance priorities:
   - Top-ranked stations should receive immediate attention
   - Health scores can be recalculated weekly for ongoing monitoring

### Business Impact

- **Proactive Maintenance**: Instead of reactive fixes, operators can now prioritize stations before problems escalate
- **Resource Optimization**: Maintenance teams can be directed to highest-priority locations
- **User Experience**: Reducing unexpected bike issues improves rider satisfaction
- **Cost Savings**: Early intervention prevents costly emergency repairs

### Limitations & Future Work

- **Data Limitation**: Analysis done at station level (bike IDs not available in this dataset)
- **External Factors**: Weather, events, and seasonal patterns not incorporated
- **Validation**: Would benefit from actual maintenance records to validate predictions

### Recommendations

1. Deploy health scoring as weekly monitoring dashboard
2. Investigate top 20 anomalous stations immediately
3. Collect and integrate actual maintenance logs for model validation
4. Consider weather data integration for more accurate predictions

In [ ]:
# Final Summary Statistics
print("="*60)
print("📊 PROJECT SUMMARY")
print("="*60)
print(f"\nTotal trips analyzed: {len(df_clean):,}")
print(f"Stations analyzed: {len(station_features_filtered)}")
print(f"")
print(f"Cluster Distribution:")
for name in station_features_filtered['cluster_name'].unique():
    count = (station_features_filtered['cluster_name'] == name).sum()
    print(f"  {name}: {count} stations")

print(f"\nAnomaly Detection:")
print(f"  Anomalous stations: {station_features_filtered['is_anomaly'].sum()}")
print(f"  Anomaly rate: {station_features_filtered['is_anomaly'].mean()*100:.1f}%")

print(f"\nHealth Status:")
for cat in ['🟢 Stable', '🟡 Warning', '🔴 Critical']:
    count = (station_features_filtered['health_category'] == cat).sum()
    pct = count / len(station_features_filtered) * 100
    print(f"  {cat}: {count} stations ({pct:.1f}%)")

print("\n" + "="*60)
print("✅ Analysis Complete!")
print("="*60)